In [ ]:
#Use this link to understnd API calls and the steps below
#https://docs.dimensions.ai/dsl/getting-started.html

In [ ]:
from tqdm.notebook import tqdm as progress
import pandas as pd
import plotly.express as px
from plotly.offline import plot
import dimcli
from dimcli.utils import *
import os, sys, time, json
from IPython.display import Image
from IPython.core.display import HTML 

In [ ]:
#Log into Dimensions
#API key is: 9F8D648F0D7E437CB1736BEBDF007F02
#!pip install dimcli -U --quiet 

print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
    import getpass
    KEY = getpass.getpass(prompt='API Key: ')  
    dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
    KEY = "9F8D648F0D7E437CB1736BEBDF007F02"
    dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()

In [ ]:
names = [
    "Michael Agus",
    "Dorothy Bainton",
    "Charles Balch",
    "Jeffrey Balser",
    "Kevin Catt",
    "Henry Chambers",
    "Richard Clark",
    "Stephanie Eisenbarth",
    "Peng Ji", 
    "Peter Fecci", 
    "Tippi Mackenzie", 
    "Purna Kashyap"
]

In [ ]:
def dimensionsPatentSearch(name):
    res = dsl.query_iterative(f"""search researchers for "{name}" return researchers""")
    # print(res.json) #list of dictionaries
    listDict = res.json['researchers']
    allIds = []
    for idv in listDict:
        allIds.append(idv['id'])
    allIds = [f'"{item}"' for item in allIds]
    allIds = '[' + ', '.join(allIds) + ']'
    print(f'Here are all the ids for {name}: ')
    print(allIds)
    
    res2 = dsl.query_iterative(f"""search researchers where id in {allIds} return researchers[id+obsolete]""")
    listDict2 = res2.json['researchers']
    allWorkingIDs = []
    for person in listDict2:
        if person['obsolete'] == 0:
            print(f'Current Working Id(s): ')
            print(person['id'])
            workingid = person['id']
            allWorkingIDs.append(person['id'])
    print(f'++++++++There are {len(allWorkingIDs)} Working IDs+++++++++++')        
    if len(allWorkingIDs) == 1:
        res3 = dsl.query_iterative(f"""search patents where researchers = "{workingid}" return patents""")
        listDict3 = res3.json['patents']
        count = 0
        finalSet = set()
        lowerSet = set()
        for l in listDict3:
            if l['title'].lower() not in lowerSet:
                finalSet.add(l['title'])
                lowerSet.add(l['title'].lower())
            else:
                count += 1
    else:
        res3 = dsl.query_iterative(f"""search patents in inventors for "{name}" return patents""")
        print(res3)
#         listDict3 = res3.json['patents']
#         count = 0
#         finalSet = set()
#         lowerSet = set()
#         for l in listDict3:
#             if l['title'].lower() not in lowerSet:
#                 finalSet.add(l['title'])
#                 lowerSet.add(l['title'].lower())
#             else:
#                 count += 1
    
    print(f'There are {count} number of duplicated patents removed')
    print(f'Final set has {len(finalSet)} number of patents')
    print()
    return (finalSet)

dimensionsPatentSearch("Charles Balch")